# Simple Example (MNIST) with TensorBoard
Sources:
- Simple example: https://ischlag.github.io/2016/06/03/simple-neural-network-in-tensorflow/
- Using TensorBoard and SummaryWriter: https://ischlag.github.io/2016/06/04/how-to-use-tensorboard/
- and http://blog.altoros.com/visualizing-tensorflow-graphs-with-tensorboard.html

### After executing these cells, launch tensorboard:
```
tensorboard --logdir=$HOME/data/tf_tut/mnist/logs
```
### And go the local url
Note: on my Mac, I was not able to load the mentioned page `http://192.168.1.6:6006`, but using localhost worked:
- http://localhost:6006/

### In Summary
- TensorBoard works with TensorFlow events files to visualize a graph and information related to its execution. 
- To generate and add log data to the TensorBoard, use the TensorFlow summary operations and the SummaryWriter. 

In [1]:
import os, sys
import tensorflow as tf

MNIST is a simple classification into 10 classes (the 10 numbers)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
HOME = os.environ['HOME']
dpath = HOME + "/data/tf_tut/mnist/MNIST_data"
mnist = input_data.read_data_sets(dpath, one_hot=True)

Extracting /Users/maximn/data/tf_tut/mnist/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/maximn/data/tf_tut/mnist/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/maximn/data/tf_tut/mnist/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/maximn/data/tf_tut/mnist/MNIST_data/t10k-labels-idx1-ubyte.gz


Let's initialize the SummaryWriter


In [3]:
log_path = dpath + "/../logs"

In [4]:
# config
batch_size = 100
learning_rate = 0.01
training_epochs = 10

Provide name-scope and variable names for easier visualization with TensorBoard

In [5]:
with tf.name_scope('input'):
    # None -> batch size can be any size, 784 -> flattened mnist image
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x-input") 
    # target 10 output classes
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y-input")

# None -> batch size can be any size, 784 -> flattened mnist image
#x = tf.placeholder(tf.float32, shape=[None, 784]) 
# target 10 output classes
#y_ = tf.placeholder(tf.float32, shape=[None, 10])

A variable in tensorflow is a value which can change. Usually, this corresponds to the parameters of the model we are going to train. In this case, the weights are according to the weight matrix of a neural network and the biases of each neurone. The shape of these variables corresponds to the size of our network.



In [6]:
# model parameters will change during training so we use tf.Variable
with tf.name_scope("weights"):
    W = tf.Variable(tf.zeros([784, 10]))
# bias
with tf.name_scope("biases"):
    b = tf.Variable(tf.zeros([10]))

Super simple model with no hidden layer, with only a softmax for the 10 classes

In [7]:
# y is our prediction
with tf.name_scope("softmax"):
    y = tf.nn.softmax(tf.matmul(x,W) + b)

Using cross entropy as the cost function

In [8]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

Predictions

In [9]:
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Optimizer operation
- tf has multiple optimizers, we'll use GD for now

In [10]:
with tf.name_scope('train'):
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy) 

In [11]:
# create a summary for our cost and accuracy
tf.scalar_summary("cost", cross_entropy)
tf.scalar_summary("accuracy", accuracy)

w_h = tf.histogram_summary("weights", W)
b_h = tf.histogram_summary("biases", b)

# merge all summaries into a single "operation" which we can execute in a session 
summary_op = tf.merge_all_summaries()

In [12]:
with tf.Session() as sess:
    # variables need to be initialized before we can use them
    sess.run(tf.initialize_all_variables())
    
    # create log writer object
    writer = tf.train.SummaryWriter(log_path, graph=tf.get_default_graph())
    
    for epoch in range(training_epochs):
        # number of batches in one epoch
        batch_count = int(mnist.train.num_examples / batch_size)
        for i in range(batch_count):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            # sess.run([train_op], feed_dict={x: batch_x, y_: batch_y})
            # perform the operations we defined earlier on batch
            _, summary = sess.run([train_op, summary_op], feed_dict={x: batch_x, y_: batch_y})
            
            # write operations summary for each iteration
            writer.add_summary(summary, epoch * batch_count + i)

        if epoch % 2 == 0: 
            print("Epoch: ", epoch )
    print("Accuracy: ", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    print("done"            )

Epoch:  0
Epoch:  2
Epoch:  4
Epoch:  6
Epoch:  8
Accuracy:  0.9026
done
